This notebook provides plots of data provided by the NY Times repository of data on coronavirus cases and deaths in the U.S. (see https://github.com/nytimes/covid-19-data, retrieved on 4/17/20)

The top part of this notebook is taken from a notebook provided at https://github.com/jsirott/covid-19-data.

The part below "Additions for the CESMII/RTG workshop" extends the plots with some interactive elements.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter

In [ ]:
df = pd.read_csv('nytimes-us-counties.csv')
kc = df[df.county == 'King'].set_index('date')
kc['delta_cases'] = kc['cases'] - kc['cases'].shift(1) 
kc['delta_deaths'] = kc['deaths'] - kc['deaths'].shift(1) 
kc.head()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,8))
ax[0].set_title('Cases')
ax[1].set_title('New cases')
subplot = kc['cases'].plot(kind='bar',logy=True,ax=ax[0])
subplot = kc['delta_cases'].plot(kind='bar',ax=ax[1])
ax[0].yaxis.set_major_formatter(ScalarFormatter())


In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,8))
ax[0].set_title('Deaths')
ax[1].set_title('New deaths')
subplot = kc['deaths'].plot(kind='bar',logy=True,ax=ax[0])
subplot = kc['delta_deaths'].plot(kind='bar',ax=ax[1])
ax[0].yaxis.set_major_formatter(ScalarFormatter())


# Additions for the CESMII/RTG workshop

The plots above show cases and deaths for Kings County in Washington state.

It would be useful to be able to select which county we want to view, as well as select by state.

To do this, let's use the ipywidgets library.

In [ ]:
import ipywidgets

To add an interactive menu for county selection:
* put the lines for plotting into a function
* include two lines to define kc based on a general county variable
* call ipywidgets.interact with the name of the function and a parameter equal to the list of counties

In [ ]:
def makeplot(county='King'):

    kc = df[(df.county==county)].set_index('date')
    kc['delta_cases'] = kc['cases'] - kc['cases'].shift(1) 
    
    fig,ax = plt.subplots(1,2,figsize=(16,8))
    ax[0].set_title('Cases')
    ax[1].set_title('New cases')
    subplot = kc['cases'].plot(kind='bar',logy=True,ax=ax[0])
    subplot = kc['delta_cases'].plot(kind='bar',ax=ax[1])
    ax[0].yaxis.set_major_formatter(ScalarFormatter())

ipywidgets.interact(makeplot,county=df.county.unique());

To add a further selection option for state in the next cell, we do the following:
* define the function with two arguments
* define kc so that its data is specified for both county and state
* explicitly declare two menu widgets (`ipywidgets.Dropdown`) whose `options` are equal to unique (and sorted) lists of county and state values
  * the default values have optionally been specified to be Los Angeles, CA
* only list counties that are in a selected state
  * this is done by "observing" the state menu via `d1.observe`
  * this function is called with a function that sets the new options for the county widget to be equal to the counties of the selected "new" state, where `change.new` is the `value` of the widget being observed

In [ ]:
def makecases(state='California',county='Los Angeles'):
    
    kc = df[(df.county == county) & (df.state == state)].set_index('date')
    kc['delta_cases'] = kc['cases'] - kc['cases'].shift(1) 
    
    fig,ax = plt.subplots(1,2,figsize=(16,8))
    ax[0].set_title('Cases')
    ax[1].set_title('New cases')
    subplot = kc['cases'].plot(kind='bar',logy=True,ax=ax[0])
    subplot = kc['delta_cases'].plot(kind='bar',ax=ax[1])
    ax[0].yaxis.set_major_formatter(ScalarFormatter())

d1 = ipywidgets.Dropdown(options= sorted(df['state'].unique()),value='California')
d2 = ipywidgets.Dropdown(options= sorted(df.loc[df['state']==d1.value,'county'].unique()),value='Los Angeles')
def changestate(change):
    d2.options=sorted(df.loc[df['state']==change.new,'county'].unique())
d1.observe(changestate,'value')

ipywidgets.interact(makecases,state=d1,county=d2);

## plotly

As one further example, we show how to make a bare-bones interactive plot with plotly.

The interactivity is now both via the menu items and via the mouse control over the generated plot.

After making the plot, try moving your mouse over the plot to see the tooltips, investigate the little menu that becomes visible, zoom in on the plot, and etc.

In [ ]:
import plotly.express as px

In [ ]:
def makecases(state='California',county='Los Angeles'):
    kc = df[(df.county == county) & (df.state == state)]
    fig = px.bar(kc,x='date',y='cases',title='Cases')
    fig.update_layout(yaxis_type="log")
    fig.show()

d1 = ipywidgets.Dropdown(options=sorted(df['state'].unique()),value='California')
d2 = ipywidgets.Dropdown(options=sorted(df.loc[df['state']=='California','county'].unique()),value='Los Angeles')
def changestate(change):
    d2.options=sorted(df.loc[df['state']==change.new,'county'].unique())
d1.observe(changestate,'value')

ipywidgets.interact(makecases,state=d1,county=d2);

In [ ]:
def makecases(state='California',county='Los Angeles'):
    kc = df[(df.county == county) & (df.state == state)]
    fig = px.bar(kc,x='date',y='deaths',title='Deaths')
    fig.show()

d1 = ipywidgets.Dropdown(options=sorted(df['state'].unique()),value='California')
d2 = ipywidgets.Dropdown(options=sorted(df.loc[df['state']=='California','county'].unique()),value='Los Angeles')
def changestate(change):
    d2.options=sorted(df.loc[df['state']==change.new,'county'].unique())
d1.observe(changestate,'value')

ipywidgets.interact(makecases,state=d1,county=d2);